In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import scipy.stats as st
import os.path
from os import path
import json

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_players_Titled = "output_data/players_Titled.csv"

country_filepath = "resources/countries.csv"

titled_players_type = ["GM","WGM","IM","WIM","FM","WFM","NM","WNM","CM","WCM"]

# Range of latitudes and longitudes
# lat_range = (-90, 90)
# lng_range = (-180, 180)

In [2]:
# function get usernames by country, pass country code and output file
# returns list of usernames for specific country and saves output file to csv
def get_usernames_country(country, output_file):
    try: 
        # ***********************************
        #
        # LIMIT run once per 24 hours to get
        # username list by country
        #
        # ***********************************

        # country code # countries list resource https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2
        # country = "RU" # RUSSIA = RU

        query_url = f"https://api.chess.com/pub/country/{country}/players"

        print(f"Fetching player list for country {country}")

        
        # Get player usernames for country
        username_response = requests.get(query_url)
        username_json = username_response.json()        
        # username_json
        
        usernames = []
        for username in username_json["players"]:
            usernames.append(username)

        player_dict = {    
            "username":usernames
        }
        chessplayers_df = pd.DataFrame(player_dict)
        chessplayers_df.to_csv(output_file, index=True,index_label="player_index")          
        
        return chessplayers_df
        
        
    except: 
        print("Unexpected error.")

In [3]:
country_df = pd.read_csv(country_filepath)
country_df

,Code,Country,File Path
0,AD,Andorra,players_AD.csv
1,AE,United Arab Emirates,players_AE.csv
2,AF,Afghanistan,players_AF.csv
3,AG,Antigua and Barbuda,players_AG.csv
4,AI,Anguilla,players_AI.csv
...,...,...,...
244,YE,Yemen,NaN
245,YT,Mayotte,NaN
246,ZA,South Africa,NaN
247,ZM,Zambia,NaN


In [4]:
def get_titledplayers_api(playertypes):
    url = "https://api.chess.com/pub/titled/"
    
    i = 0        
    players_data = []
    
    for playertype in playertypes:
        print(f"Index {str(i)} fetching player list for titled player type {playertype}")
    
        # Build query URL
        query_url = f"{url}{playertype}"

        # increment index counter
        i += 1

        # Get weather data    
        playertype_response = requests.get(query_url)
        playertype_json = playertype_response.json()
        players_data.append(playertype_json)        
        time.sleep(1)    
        
    return players_data

In [5]:
def get_playerdata_api(player_usernames):
    url = "https://api.chess.com/pub/player/"
    
    i = 0        
    player_data = []
    
    for player_username in player_usernames:
        print(f"Index {str(i)} fetching player data for player {player_username}")
    
        # Build query URL
        query_url = f"{url}{player_username}"

        # increment index counter
        i += 1

        # Get player data    
        try:
            player_response = requests.get(query_url)
            player_json = player_response.json()
            player_data.append(player_json)        
            time.sleep(1)    
        except:
            print("unexpected error")
        
        # if i > 10:
        #    break
    return player_data

In [6]:
def get_player_dataframe(playerdata):                
    player_id = []
    player_username = []
    title = []
    followers = []
    country = []
    location = []
    last_online = []
    joined = []
    status = []
    is_streamer = []

    i = 0

    for player in playerdata: 
        try:            
            username = player["username"]
            print(f"Index {i} processing username {username}")
            player_id.append(player["player_id"])
            player_username.append(username)
            if "title" in player.keys():
                title.append(player["title"])
            else:
                title.append("")          
            followers.append(player["followers"])        
            country.append(player["country"].replace("https://api.chess.com/pub/country/","")),
            if "location" in player.keys():
                location.append(player["location"])        
            else:
                location.append("")                
            last_online.append(player["last_online"])
            joined.append(player["joined"])
            status.append(player["status"])
            if "is_streamer" in player.keys():
                is_streamer.append(player["is_streamer"])                       
            else:     
                is_streamer.append("")
        except:
            print("unexpected error")

        i += 1 

    player_dict = {
        "player_id":player_id,
        "username":player_username,
        "title":title,
        "followers":followers,
        "country":country,
        "location":location,
        "last_online":last_online,
        "joined":joined,
        "status":status,
        "is_streamer":is_streamer
    }
    chessplayer_details_df = pd.DataFrame(player_dict)

    # chessplayer_df.to_csv(output_data_players, index=True,index_label="player_index") 
    
    return chessplayer_details_df

In [7]:
for index, row in country_df.iterrows():        
    try:          
        country_name = row["Country"]
        country_code = row["Code"]
        print(f"Fetching usernames for {country_name} ({country_code})")
        output_file_country = f"output_data/players_{country_code}.csv"
        
        get_usernames_country(country_code,output_file_country)
        time.sleep(1)  
        
    except: 
        print("unepxected error/skipping country")

Fetching usernames for Andorra (AD)
Fetching player list for country AD
Fetching usernames for United Arab Emirates (AE)
Fetching player list for country AE
Fetching usernames for Afghanistan (AF)
Fetching player list for country AF
Fetching usernames for Antigua and Barbuda (AG)
Fetching player list for country AG
Fetching usernames for Anguilla (AI)
Fetching player list for country AI
Fetching usernames for Albania (AL)
Fetching player list for country AL
Fetching usernames for Armenia (AM)
Fetching player list for country AM
Fetching usernames for Angola (AO)
Fetching player list for country AO
Fetching usernames for Antarctica (AQ)
Fetching player list for country AQ
Unexpected error.
Fetching usernames for Argentina (AR)
Fetching player list for country AR
Fetching usernames for American Samoa (AS)
Fetching player list for country AS
Fetching usernames for Austria (AT)
Fetching player list for country AT
Fetching usernames for Australia (AU)
Fetching player list for country AU
Fet

Fetching usernames for Iraq (IQ)
Fetching player list for country IQ
Fetching usernames for Iran (Islamic Republic of) (IR)
Fetching player list for country IR
Fetching usernames for Iceland (IS)
Fetching player list for country IS
Fetching usernames for Italy (IT)
Fetching player list for country IT
Fetching usernames for Jersey (JE)
Fetching player list for country JE
Fetching usernames for Jamaica (JM)
Fetching player list for country JM
Fetching usernames for Jordan (JO)
Fetching player list for country JO
Fetching usernames for Japan (JP)
Fetching player list for country JP
Fetching usernames for Kenya (KE)
Fetching player list for country KE
Fetching usernames for Kyrgyzstan (KG)
Fetching player list for country KG
Fetching usernames for Cambodia (KH)
Fetching player list for country KH
Fetching usernames for Kiribati (KI)
Fetching player list for country KI
Fetching usernames for Comoros (KM)
Fetching player list for country KM
Fetching usernames for Saint Kitts and Nevis (KN)
F

Fetching usernames for Eswatini (SZ)
Fetching player list for country SZ
Fetching usernames for Turks and Caicos Islands (TC)
Fetching player list for country TC
Fetching usernames for Chad (TD)
Fetching player list for country TD
Fetching usernames for French Southern Territories (TF)
Fetching player list for country TF
Unexpected error.
Fetching usernames for Togo (TG)
Fetching player list for country TG
Fetching usernames for Thailand (TH)
Fetching player list for country TH
Fetching usernames for Tajikistan (TJ)
Fetching player list for country TJ
Fetching usernames for Tokelau (TK)
Fetching player list for country TK
Unexpected error.
Fetching usernames for Timor-Leste (TL)
Fetching player list for country TL
Fetching usernames for Turkmenistan (TM)
Fetching player list for country TM
Fetching usernames for Tunisia (TN)
Fetching player list for country TN
Fetching usernames for Tonga (TO)
Fetching player list for country TO
Fetching usernames for Turkey (TR)
Fetching player list f

In [ ]:
usernames_ = get_usernames_country(country_code,output_file_country)

In [ ]:
playerslist_data = get_titledplayers_api(titled_players_type)

In [20]:
playerlist_index = 0
i = 0

player_usernames = []

for playerlist in playerslist_data:    
    for player in playerlist["players"]:        
        player_usernames.append(player)
        
    playerlist_index += 1

```{
  "avatar": "https://images.chesscomfiles.com/uploads/v1/user/14775486.9072453d.200x200o.50bb2d10a7d5.jpeg",
  "player_id": 14775486,
  "@id": "https://api.chess.com/pub/player/140mariam",
  "url": "https://www.chess.com/member/140mariam",
  "name": "Mariam Avetisyan",
  "username": "140mariam",
  "title": "WFM",
  "followers": 56,
  "country": "https://api.chess.com/pub/country/AM",
  "last_online": 1609780423,
  "joined": 1385829847,
  "status": "premium",
  "is_streamer": false
}
```

In [19]:
# check if output_data_file exists already, if it doesn't exist, do api call to get weather data
if (path.exists(output_data_players)):
    # ****************
    # pull prevously generated and saved output data from csv
    chessplayer_df = pd.read_csv(output_data_players)
else:    
    # run api calls for each chess.com username
    # this step is skipped if the output_data_file already exists
    # !!!!! DELETE CSV OUTPUT FILE to generate new playerdata    
    playerdata = get_playerdata_api(player_usernames)
    
    
    
    

NameError: name 'output_data_players' is not defined

In [8]:
chessplayers_country_df

NameError: name 'chessplayers_country_df' is not defined

In [ ]:
## Get sample of users from csv file of one country
country = "BR"
players_country_file = f"output_data/players_{country}.csv"

chessplayers_country_df = pd.read_csv(players_country_file)
chessplayers_count = len(chessplayers_country_df)    
sample_count = int(round(chessplayers_count * .15,0))
#
print(f"{sample_count} is the sample size (15%) of {country} population ({chessplayers_count})\n")
data = np.random.randint(0, chessplayers_count, size=sample_count)

sample_country_usernames = []
for index in data:    
    username = chessplayers_country_df.iloc[index]["username"]    
    sample_country_usernames.append(username)
    

playerdata = get_playerdata_api(sample_country_usernames)


24327 is the sample size (15%) of BR population (162177)

Index 0 fetching player data for player digolll
Index 1 fetching player data for player lbmello
Index 2 fetching player data for player sasah345
Index 3 fetching player data for player flavio16t
Index 4 fetching player data for player fernandauau
Index 5 fetching player data for player johshelby
Index 6 fetching player data for player cjkastec
Index 7 fetching player data for player emanuelduarte
Index 8 fetching player data for player mystica89
Index 9 fetching player data for player gruncarlos
Index 10 fetching player data for player julys-arq
Index 11 fetching player data for player juan_vereda
Index 12 fetching player data for player drborsato
Index 13 fetching player data for player saulobraga1
Index 14 fetching player data for player cml7588
Index 15 fetching player data for player vtrarc
Index 16 fetching player data for player adpmelhor
Index 17 fetching player data for player gu342
Index 18 fetching player data for play

Index 156 fetching player data for player jogonca
Index 157 fetching player data for player iidoog
Index 158 fetching player data for player alyssonr
Index 159 fetching player data for player edudonidabgrsbr
Index 160 fetching player data for player jose4272
Index 161 fetching player data for player a_asoares
Index 162 fetching player data for player edmarqueschess
Index 163 fetching player data for player lcn6
Index 164 fetching player data for player rafaelfss
Index 165 fetching player data for player rschess1999
Index 166 fetching player data for player bdigo7
Index 167 fetching player data for player lucasironside
Index 168 fetching player data for player potato_guxta
Index 169 fetching player data for player yannventura
Index 170 fetching player data for player rodrigomussatto
Index 171 fetching player data for player rogernote
Index 172 fetching player data for player nando0039
Index 173 fetching player data for player eliaskmaluuko
Index 174 fetching player data for player dutra

Index 311 fetching player data for player tylerqueenston
Index 312 fetching player data for player 1gms1
Index 313 fetching player data for player nanandaime
Index 314 fetching player data for player samuelegea
Index 315 fetching player data for player smith198931
Index 316 fetching player data for player dentinhogho
Index 317 fetching player data for player joaomarendaz
Index 318 fetching player data for player vidzera
Index 319 fetching player data for player pdttroll
Index 320 fetching player data for player vanynha
Index 321 fetching player data for player mathega
Index 322 fetching player data for player carloshe0
Index 323 fetching player data for player edivan76
Index 324 fetching player data for player dudagoulart76
Index 325 fetching player data for player jpkynusty
Index 326 fetching player data for player raibolt
Index 327 fetching player data for player felipebnas
Index 328 fetching player data for player caiovf
Index 329 fetching player data for player antonopoulosj
Index 

Index 466 fetching player data for player chmlima
Index 467 fetching player data for player macacofedido
Index 468 fetching player data for player guilhermepimenta
Index 469 fetching player data for player manadruz
Index 470 fetching player data for player dudamorari2020
Index 471 fetching player data for player kikorangelsoriano
Index 472 fetching player data for player luancelote
Index 473 fetching player data for player jay3886
Index 474 fetching player data for player jeffersonsena
Index 475 fetching player data for player glabellemaria
Index 476 fetching player data for player vitaosemluizasonza
Index 477 fetching player data for player dr_chessnnibal
Index 478 fetching player data for player gggarcia96
Index 479 fetching player data for player patrickfaulstich
Index 480 fetching player data for player bobcavalcante
Index 481 fetching player data for player talita30
Index 482 fetching player data for player lucas240
Index 483 fetching player data for player marcos1234silva
Index 4

Index 620 fetching player data for player matheusfoganholi
Index 621 fetching player data for player rodrigosmenezes
Index 622 fetching player data for player moratojv
Index 623 fetching player data for player joaoatno
Index 624 fetching player data for player teixeira1970
Index 625 fetching player data for player montevsk
Index 626 fetching player data for player 07mazinho
Index 627 fetching player data for player itoxiczzz
Index 628 fetching player data for player weslleycaruzo
Index 629 fetching player data for player lufeexx
Index 630 fetching player data for player matheocc
Index 631 fetching player data for player mariapaularegal
Index 632 fetching player data for player adleuson
Index 633 fetching player data for player morterax
Index 634 fetching player data for player alicerj
Index 635 fetching player data for player bentobm
Index 636 fetching player data for player rychlewskigomes
Index 637 fetching player data for player joaorochabh
Index 638 fetching player data for player 

...

Index 1996 fetching player data for player rob553
Index 1997 fetching player data for player wilson198
Index 1998 fetching player data for player monaro706
Index 1999 fetching player data for player andrezinho790
Index 2000 fetching player data for player jairborges
Index 2001 fetching player data for player arthur1238038
Index 2002 fetching player data for player p0etal0k0
Index 2003 fetching player data for player knopix31
Index 2004 fetching player data for player bardellalove
Index 2005 fetching player data for player lfmelo
Index 2006 fetching player data for player gianbueno
Index 2007 fetching player data for player o_pianista95
Index 2008 fetching player data for player ays0712
Index 2009 fetching player data for player davificher
Index 2010 fetching player data for player favaretto19
Index 2011 fetching player data for player fabricioou
Index 2012 fetching player data for player rigby_tt
Index 2013 fetching player data for player maria_lisboa
Index 2014 fetching player data fo

Index 2148 fetching player data for player dedezeira
Index 2149 fetching player data for player joaovwinklam
Index 2150 fetching player data for player gadbkddd
Index 2151 fetching player data for player eribeiro73
Index 2152 fetching player data for player panicdisco675
Index 2153 fetching player data for player diogosbastos
Index 2154 fetching player data for player pedrurius
Index 2155 fetching player data for player cauah7976
Index 2156 fetching player data for player jgp47
Index 2157 fetching player data for player narrowpenguim
Index 2158 fetching player data for player xequemate94
Index 2159 fetching player data for player luigivaccaro10
Index 2160 fetching player data for player james_arthur14
Index 2161 fetching player data for player fecaramico
Index 2162 fetching player data for player lucaspozzolini
Index 2163 fetching player data for player philipesouza7
Index 2164 fetching player data for player guga222
Index 2165 fetching player data for player vapovap0
Index 2166 fetchi

Index 2300 fetching player data for player asfrald
Index 2301 fetching player data for player marcoscaridade
Index 2302 fetching player data for player fidio
Index 2303 fetching player data for player towerofdestiny
Index 2304 fetching player data for player patrimario
Index 2305 fetching player data for player miguel181103
Index 2306 fetching player data for player ferbaaa
Index 2307 fetching player data for player firosamy
Index 2308 fetching player data for player lscabello
Index 2309 fetching player data for player missaodenis
Index 2310 fetching player data for player miguelson95
Index 2311 fetching player data for player pedroapenas
Index 2312 fetching player data for player perfectpiano
Index 2313 fetching player data for player rebelc
Index 2314 fetching player data for player capitaots
Index 2315 fetching player data for player suegr27
Index 2316 fetching player data for player alvinfreak
Index 2317 fetching player data for player math1671
Index 2318 fetching player data for p

Index 2452 fetching player data for player aryelmandrake
Index 2453 fetching player data for player nullchess999
Index 2454 fetching player data for player marcosbc31
Index 2455 fetching player data for player taufik5673
Index 2456 fetching player data for player xinoumatheus
Index 2457 fetching player data for player tadeucoelho04
Index 2458 fetching player data for player uiu_apenas
Index 2459 fetching player data for player mateusconstantino
Index 2460 fetching player data for player lucianovictorino11
Index 2461 fetching player data for player saastheo
Index 2462 fetching player data for player walberjunior
Index 2463 fetching player data for player jftb17
Index 2464 fetching player data for player meliko10
Index 2465 fetching player data for player pvcorradini
Index 2466 fetching player data for player rotsaad
Index 2467 fetching player data for player jonasosilva
Index 2468 fetching player data for player guilherme_rosito
Index 2469 fetching player data for player mufasa1616
Inde

Index 2604 fetching player data for player swh_maik
Index 2605 fetching player data for player pedrotiago_87
Index 2606 fetching player data for player brunocristiano
Index 2607 fetching player data for player renatommeireles
Index 2608 fetching player data for player lonewolf1473
Index 2609 fetching player data for player dfbrandao
Index 2610 fetching player data for player mahitz
Index 2611 fetching player data for player fnogueira1
Index 2612 fetching player data for player kraos3
Index 2613 fetching player data for player renanzinhojoquenpou
Index 2614 fetching player data for player xcalisorde
Index 2615 fetching player data for player merinovisk
Index 2616 fetching player data for player ruukasu07
Index 2617 fetching player data for player dutrals
Index 2618 fetching player data for player rogercsx
Index 2619 fetching player data for player malado22
Index 2620 fetching player data for player matt_mattson
Index 2621 fetching player data for player hrsilveira
Index 2622 fetching pl

In [ ]:
player_BR_df = get_player_dataframe(playerdata)

In [ ]:
player_BR_df

In [74]:
output_data_players_BR = 'output_data/player_details_BR.csv'
player_BR_df.to_csv(output_data_players_AD, index=True,index_label="player_index") 